# python->onnx->ncnn（YOLOv5为例）

## python->onnx

### 获取YOLOv5并安装所需环境

In [ ]:
# 克隆yolov5
!git clone https://github.com/ultralytics/yolov5  
# cd到yolov5路径下
%cd yolov5
# 安装所需环境
%pip install -qr requirements.txt

### 导入torch并检测是否为GPU环境

In [ ]:
# 导入torch
import torch
# 查看是否为GPU环境
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

### 安装转onnx格式的导出环境

In [ ]:
!pip install onnx>=1.7.0  # for ONNX export
!pip install coremltools==4.0  # for CoreML export
!pip install onnx-simplifier

### 使用yolov5自带的export.py进行转换

In [ ]:
!python /content/yolov5/models/export.py --weights /content/drive/MyDrive/yolov5s-bug.pt --img 640 --batch 1

### 对生成的onnx进行优化

In [ ]:
!python -m onnxsim /content/drive/MyDrive/yolov5s-bug.onnx /content/drive/MyDrive/yolov5s-bug-sim.onnx

## onnx->ncnn

### 方案1：使用编译环境

#### 安装g++编译环境

In [ ]:
!sudo apt-get install autoconf automake libtool curl make g++ unzip

#### 克隆protobuf并配置make环境

In [ ]:
cd /content/
!git clone https://github.com/protocolbuffers/protobuf.git
%cd /content/protobuf
!git submodule update --init --recursive
!./autogen.sh
!./configure
!make
!make check
!sudo make install
!sudo ldconfig

#### 配置腾讯ncnn编译环境

In [ ]:
%cd /content
!git clone https://github.com/Tencent/ncnn.git
%cd /content/ncnn
!mkdir -p build
%cd /content/ncnn/build
!cmake -DNCNN_VULKAN=OFF ..  #vulkan是针对gpu的，如果想要ncnn能调用gpu做推理，那么选项需要打开，设置为ON。
!make -j4  #开始编译

#### cd到onnx2ncnn目录下，使用其进行转化ncnn

In [ ]:
%cd /content/ncnn/build/tools/onnx/
!./onnx2ncnn /content/drive/MyDrive/yolov5s-bug-sim.onnx /content/drive/MyDrive/model.param /content/drive/MyDrive/model.bin


出现以下错误为正常现象，因为YOLOv5的Focus模块需要修改，并进行定义
```
Unsupported slice step !
Unsupported slice step !
Unsupported slice step !
Unsupported slice step !
Unsupported slice step !
Unsupported slice step !
Unsupported slice step !
Unsupported slice step !
```



#### 注意：YOLOv5在此到下一步之前需要对.param文件进行更改

```
7767517
176 200
Input                    images                   0 1 images
YoloV5Focus              focus                    1 1 images 167
Convolution              Conv_41                  1 1 167 168 0=32 1=3 4=1 5=1 6=3456
Swish                    Mul_43                   1 1 168 170
Convolution              Conv_44                  1 1 170 171 0=64 1=3 3=2 4=1 5=1 6=18432
Swish                    Mul_46                   1 1 171 173
Split                    splitncnn_0              1 2 173 173_splitncnn_0 173_splitncnn_1
Convolution              Conv_47                  1 1 173_splitncnn_1 174 0=32 1=1 5=1 6=2048
Swish                    Mul_49                   1 1 174 176
Split                    splitncnn_1              1 2 176 176_splitncnn_0 176_splitncnn_1
Convolution              Conv_50                  1 1 176_splitncnn_1 177 0=32 1=1 5=1 6=1024
```



#### cd到ncnnoptimize目录下，使用其进行优化

In [ ]:
%cd /content/ncnn/build/tools/
!./ncnnoptimize /content/drive/MyDrive/yolov5s.param  /content/drive/MyDrive/yolov5s.bin /content/drive/MyDrive/new.param /content/drive/MyDrive/new.bin 65536

create_custom_layer YoloV5Focus
model has custom layer, shape_inference skipped
model has custom layer, estimate_memory_footprint skipped


### 方案2：使用转换网站
[转换网站](https://convertmodel.com/)

## 参考文献
[YOLO5实现安卓APP目标检测](https://blog.csdn.net/qq_29750461/article/details/114818608)

[yolov5 pt转onnx_yolov5＞onnx&gt;ncnn＞安卓apk](https://blog.csdn.net/weixin_39808893/article/details/111136990)

[详细记录u版YOLOv5目标检测ncnn实现](https://zhuanlan.zhihu.com/p/275989233)

[指尖上的YOLOv5](https://my.oschina.net/u/4883419/blog/4942998)

[YOLO5实现安卓APP目标检测](https://blog.csdn.net/qq_29750461/article/details/114818608)

[模型部署之 ONNX ONNXRuntime](https://www.jianshu.com/p/476478c17b8e)

[pytorch 转换onnx_Pytorch转NCNN的流程记录](https://blog.csdn.net/weixin_39752434/article/details/112174118?utm_medium=distribute.pc_relevant.none-task-blog-baidujs_title-1&spm=1001.2101.3001.4242)

[onnxruntime模型部署流程](https://zhuanlan.zhihu.com/p/261307813)